In [1]:
import os
from google.oauth2 import service_account  # type: ignore
import googleapiclient.discovery  # type: ignore
from google.cloud import bigquery
import pandas as pd
import requests


In [2]:
# client = bigquery.Client()

# query = '''
# WITH transfers AS (
# SELECT 
#   block_timestamp, 
#   from_address, 
#   to_address, 
#   token_address, 
#   value  
# FROM `bigquery-public-data.crypto_ethereum.token_transfers`  
# WHERE TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-07-22"))
# SELECT
#   t.block_timestamp,
#   t.from_address,
#   t.to_address,
#   t.value,
#   tok.name,
#   tok.symbol,
#   tok.decimals,
#   tok.total_supply
# FROM transfers t
# LEFT JOIN `bigquery-public-data.crypto_ethereum.tokens` tok ON t.token_address = tok.address
# '''

# query_job = client.query(query)

# df = query_job.to_dataframe()
# df.to_csv('data/transactions.csv')
# df.head()

Para ahorrar capacidad computacional, he descargado la query en un csv y lo llamaré a este para optimizar el tiempo de análisis

## Voy a remover la columna total_supply porque tiene valores extraños y no confio en ellos para continuar trabajando con ellos (parecen no estar actualizados).
Una buena opción sería conseguir los datos de la oferta total de los tokens, o al menos tener un booleano que permita indicar cuales tienen capped supply. Esto sería útil para medir qué proporción de la oferta total de cada token se está transaccionando en esos momentos.

In [3]:
df = pd.read_csv('data/transactions.csv')
df = df[df.columns[1:-1]]
df['decimals'] = pd.to_numeric(df['decimals'], errors='coerce')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['value'] = df['value']/ (10**df['decimals'])
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
df['tx_month'] = df['block_timestamp'].dt.month
df['tx_year'] = df['block_timestamp'].dt.year
df.head()

/var/folders/k4/6hfj9vfx0rvbk2skj_ydz9600000gn/T/ipykernel_37471/4137274742.py:1: DtypeWarning: Columns (4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/transactions.csv')


,block_timestamp,from_address,to_address,value,name,symbol,decimals,tx_month,tx_year
0,2024-07-22 00:00:11+00:00,0x06fd4ba7973a0d39a91734bbc35bc2bcaa99e3b0,0x28c6c06298d514db089934071355e5743bf21d60,3.000000e+05,COTI Token,COTI,18.0,7,2024
1,2024-07-22 00:00:11+00:00,0x0000000000000000000000000000000000000000,0x42df00ac2e0000005486ac375a00b80000d12758,2.879000e+03,Metronome,MET,18.0,7,2024
2,2024-07-22 00:00:11+00:00,0x79320fad5780ec2109b2242b20f120ee16b5dafb,0x28c6c06298d514db089934071355e5743bf21d60,3.900000e+07,SHIBA INU,SHIB,18.0,7,2024
3,2024-07-22 00:00:11+00:00,0xa9c61fe59b5702b1d382fd1d5e495887ff34c21d,0x28c6c06298d514db089934071355e5743bf21d60,4.660659e+03,ChainLink Token,LINK,18.0,7,2024
4,2024-07-22 00:00:23+00:00,0x8670b29f42089019747b24f8d6965afa3eaf90ec,0x60d6649f0b58445d39d98aaf29d5de590111f67c,2.500000e+06,SHIBA INU,SHIB,18.0,7,2024


In [4]:
df_from_tx_count = pd.DataFrame(df.groupby(['from_address', 'tx_year', 'tx_month']).size().reset_index(name='monthly_sent_tx'))
df_from_tx_count_mean = pd.DataFrame(df_from_tx_count.groupby(['from_address'])['monthly_sent_tx'].mean().reset_index(name='avg_monthly_sent_tx'))
df_from_tx_count_mean = df_from_tx_count_mean.rename(columns={'from_address':'address'})
df_from_tx_count_mean.head()

,address,avg_monthly_sent_tx
0,0x0000000000000000000000000000000000000000,83119.0
1,0x0000000000000000000000000000000000000001,4.0
2,0x0000000000000000000000000000000000029e2c,1.0
3,0x0000000000000000000000000000000000029f3f,1.0
4,0x000000000000003607fce1ac9e043a86675c5c2f,3.0


In [5]:
df_to_tx_count = pd.DataFrame(df.groupby(['to_address', 'tx_year', 'tx_month']).size().reset_index(name='monthly_received_tx'))
df_to_tx_count_mean = df_to_tx_count.groupby(['to_address'])['monthly_received_tx'].mean().reset_index(name='avg_monthly_received_tx')
df_to_tx_count_mean = df_to_tx_count_mean.rename(columns = {'to_address': 'address'})
df_to_tx_count_mean.head()

,address,avg_monthly_received_tx
0,0x0000000000000000000000000000000000000000,13918.0
1,0x0000000000000000000000000000000000000001,11.0
2,0x0000000000000000000000000000000000000fee,14.0
3,0x000000000000000000000000000000000000dead,1316.0
4,0x0000000000000000000000000000000000026a63,1.0


In [6]:
#Get relative value per token sent to another address
df_value_sent = pd.DataFrame(df.groupby(['from_address', 'symbol'])['value'].sum().reset_index(name='total_value_sent_per_token'))
token_totals = df_value_sent.groupby(['symbol'])['total_value_sent_per_token'].sum().reset_index(name='total_sent_per_symbol')
df_value_sent = pd.merge(df_value_sent, token_totals, on='symbol')
df_value_sent['relative_size_sent'] = df_value_sent['total_value_sent_per_token'] / df_value_sent['total_sent_per_symbol']
df_value_sent['relative_size_sent'] = df_value_sent['relative_size_sent'].fillna(0)
df_value_sent = df_value_sent.rename(columns={'from_address': 'address'})

#Get relative received per token sent to another address
df_value_received = pd.DataFrame(df.groupby(['to_address', 'symbol'])['value'].sum().reset_index(name='total_value_received_per_token'))
token_totals_received = df_value_received.groupby(['symbol'])['total_value_received_per_token'].sum().reset_index(name='total_received_per_symbol')
df_value_received = pd.merge(df_value_received, token_totals_received, on='symbol')
df_value_received['relative_size_received'] = df_value_received['total_value_received_per_token'] / df_value_received['total_received_per_symbol']
df_value_received['relative_size_received'] = df_value_received['relative_size_received'].fillna(0)
df_value_received = df_value_received.rename(columns={'to_address': 'address'})


#merge dfs and calculate net difference
df_values_grouped = pd.merge(df_value_sent, df_value_received, on=['address', 'symbol'])
df_values_grouped['relative_net_tx'] = df_values_grouped['relative_size_received'] - df_values_grouped['relative_size_sent']

#group and get the mean value of each address (net senders or receivers)
df_net_tx = pd.DataFrame(df_values_grouped.groupby(['address'])['relative_net_tx'].mean().reset_index(name='relative_net_tx'))
df_net_tx.head()


,address,relative_net_tx
0,0x0000000000000000000000000000000000000000,-4.680780e-02
1,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,-4.263411e-08
2,0x000000000000c9b3e2c3ec88b1b4c0cd853f4321,2.694309e-07
3,0x000000000001579c68832e3b57cbf6da116fa987,4.940341e-09
4,0x00000000000ba9cd9f5175108141a82b6c24d727,3.734038e-08


In [7]:
df_sent_to_add0 = pd.DataFrame(df[df['to_address'] == '0x0000000000000000000000000000000000000000'].groupby(['from_address']).size().reset_index(name='tx_sent_to_add0'))
df_received_from_add0 = pd.DataFrame(df[df['from_address'] == '0x0000000000000000000000000000000000000000'].groupby(['to_address']).size().reset_index(name='tx_received_from_add0'))
df_add0_interactions = pd.merge(df_sent_to_add0, df_received_from_add0, left_on='from_address', right_on='to_address')
df_add0_interactions['net_interactions_w_add0'] = df_add0_interactions['tx_received_from_add0'] - df_add0_interactions['tx_sent_to_add0']
df_add0_interactions = df_add0_interactions[['from_address', 'net_interactions_w_add0']]
df_add0_interactions = df_add0_interactions.rename(columns={'from_address': 'address'})
df_add0_interactions

,address,net_interactions_w_add0
0,0x0000000000000000000000000000000000000000,0
1,0x000000000000c9b3e2c3ec88b1b4c0cd853f4321,47
2,0x00000000009e50a7ddb7a7b0e2ee6604fd120e49,-39
3,0x0000000000a84d1a9b0063a910315c7ffa9cd248,-2
4,0x00000000032962b51589768828ad878876299e14,2
...,...,...
1374,0xfee5be290d4a1a50caff611389ba2120a4095d78,0
1375,0xfeef38648754537069bbf8ae18476627eb413297,0
1376,0xff1a131e45958296cfcd4ccd0cbda78e69040654,-1
1377,0xff2ad0f6a8c303ac005338aa841b4f7f55415095,0


In [8]:
#join on address
dataframes = [df_from_tx_count_mean, df_to_tx_count_mean, df_net_tx, df_add0_interactions]

final_df = pd.DataFrame()
for dataframe in dataframes:
    if final_df.shape[1] == 0:
        final_df = dataframe
    else:
        final_df = final_df.merge(dataframe, on='address', how='outer')


In [9]:
final_df = final_df.fillna(0)

In [10]:
#get address labels
#repository here: https://github.com/dawsbot/eth-labels?tab=readme-ov-file
url = "https://eth-labels-production.up.railway.app/accounts"

response = requests.get(url)

address_labels = response.json()

In [11]:
df_labels = pd.DataFrame(address_labels)
df_labels = df_labels[df_labels['chainId'] == 1][['address', 'label']]
df_labels.head()

,address,label
0,0x0000000000000000000000000000000000000000,blocked
1,0x0000000000000000000000000000000000000000,burn
2,0x0000000000000000000000000000000000000000,genesis
3,0x0000000000000000000000000000000000000001,blocked
4,0x0000000000000000000000000000000000000001,burn


In [15]:
final_df = final_df.merge(df_labels, on='address', how='left')

In [20]:
print(final_df.groupby(['label']).size().sort_values(ascending=False))

label
sushiswap           310
mev-bot             110
take-action         100
balancer             55
bancor               49
                   ... 
pooltogether          1
bitmart               1
redacted-cartel       1
reflexer-finance      1
bittrex               1
Length: 169, dtype: int64
